In [1]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

### input and output

In [2]:
###input
ymdb_output_final = '../../../Data/ymdb/ymdb_final_result.xlsx'
mnxreac_smile_atom_mapping_rules_file_path = '../../../Data/rules/MNXreaction_smiles_atommap_rules_filter.csv'
target_smiles_path = '../../../Results/not_lipid/target_smiles_not_lipid_complex.pickle'
target_model_smiles_path = '../../../Results/not_lipid/yeast_met_not_lipid.pickle'
yeast_model_path = '../../../Data/model/yeast-GEM-final.csv'
total_met_inchikey0_file = '../../../Data/total_met_inchikey0.pickle'
uptake_met_path = '../../../Data/ymdb/ymdb_uptake.csv'
cut_off = 0.3
num = 70
num_process = 60
cut_off_path = f'../../../Results/not_lipid/top{num}_{cut_off}/'
sce_gene_clean_ec = '../../../Data/Saccharomyces_cerevisiae_teacher_maxsep.csv'
DeepEC_path = '../../../Data/Sce_DeepECv2.txt'
sce_uniprot_path = '../../../Data/uniprotkb_organism_id_559292_2023_11_08.tsv'
yeast870_path =  '../../../Data/model/yeast-GEM.yml'
miss_met_id_file = '../../../Data/model/miss_met_id.json'

###output
retrosys_smiles_calculate_similarity_file_path_all = '../../../Results/not_lipid/retrosys_smiles_calculate_similarity_0.3.json'

retrosys_smiles_calculate_similarity_file_path = cut_off_path + f'retrosys_smiles_calculate_similarity_top{num}_{cut_off}.json'
retrosys_smiles_calculate_similarity_filter_file_path = cut_off_path + f'retrosys_smiles_calculate_similarity_filter_top{num}_{cut_off}.json'
rxndb_path = cut_off_path + f'RXNdb_top{num}_{cut_off}/'
failedrxn_path = cut_off_path + f'failedrxn_top{num}_{cut_off}.pickle' 
rxndb_all_path = cut_off_path + f'RXNdb_all_top{num}_{cut_off}.json'
# rxndb_all_path = cut_off_path + f'RXNdb_all_top{num}_{cut_off}_all.csv'
rxndb_drop_path = cut_off_path + f'RXNdb_drop_top{num}_{cut_off}/'
rxndb_met_max_score_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}.csv'
rxndb_met_max_score_uptake_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}_uptake.csv'
rxndb_to_model_path = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}.csv'
YMDB_success_met_smile_file = cut_off_path + f'YMDB_success_met_smile_top{num}_{cut_off}.pickle'
YMDB_success_met_smile_uptake_file = cut_off_path + f'YMDB_success_met_smile_uptake_top{num}_{cut_off}.pickle'
YMDB_fail_met_smile_file = cut_off_path + f'YMDB_fail_met_smile_top{num}_{cut_off}.pickle'
YMDB_fail_met_smile_uptake_file = cut_off_path + f'YMDB_fail_met_smile_uptake_top{num}_{cut_off}.pickle'
rxndb_GPR_to_model_path = cut_off_path + f'rxndb_GPR_to_model_top{num}_{cut_off}.csv'
rxndb_met_max_score_met_annotation_file = cut_off_path + f'rxndb_met_max_score_met_annotation_top{num}_{cut_off}.csv'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
yeast8_reaction_in_rxndb_json = cut_off_path + f'yeast8_reaction_in_rxndb_top{num}_{cut_off}.json'




In [3]:
#check if the output folder exists
if not os.path.exists(cut_off_path):
    os.makedirs(cut_off_path)
if not os.path.exists(rxndb_path):
    os.makedirs(rxndb_path)
if not os.path.exists(rxndb_drop_path):
    os.makedirs(rxndb_drop_path)

In [4]:
yeast_total_smiles = process_yeast_smiles(target_model_smiles_path)
# yeast_total_smiles = [neutralize_charge(x) for x in yeast_total_smiles]
yeast_total_smiles = [x for x in yeast_total_smiles if Chem.MolFromSmiles(x)]
retrorules = pd.read_csv(mnxreac_smile_atom_mapping_rules_file_path)
rules_product_smiles_lst = process_retrorules(retrorules,s='product_smiles')
rules_product_smarts_lst = process_retrorules(retrorules,s='ProductSMARTs')


total_smiles 4197
total_smiles 4196
total_smiles 4196
total_smiles 2563


20210it [00:01, 12275.84it/s]


8254


20210it [00:01, 10233.88it/s]

11257


In [5]:
retrosys_smiles_calculate_similarity_pd = pd.read_json(retrosys_smiles_calculate_similarity_file_path_all)
retrosys_smiles_calculate_similarity_pd = get_most_similar_smiles(retrosys_smiles_calculate_similarity_pd,num)
retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path)

In [6]:
# retrosys_smiles_calculate_similarity = process_rules_with_multiprocessing(rules_product_smiles_lst, yeast_total_smiles,num_processes=num_process,cutoff=cut_off)

# retrosys_smiles_calculate_similarity_pd = pd.DataFrame(retrosys_smiles_calculate_similarity)
# retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path_all)

# # choose top num similar metabolites
# retrosys_smiles_calculate_similarity_pd = pd.read_json(retrosys_smiles_calculate_similarity_file_path_all)
# retrosys_smiles_calculate_similarity_pd = get_most_similar_smiles(retrosys_smiles_calculate_similarity_pd,num)
# retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path)

### get rxndb

In [7]:
retrorules = merge_smiles_similarity_rule(retrorules, retrosys_smiles_calculate_similarity_pd)


100%|██████████| 20210/20210 [00:58<00:00, 342.86it/s]


In [8]:
filter_smiles_muti(retrorules,retrosys_smiles_calculate_similarity_filter_file_path,num_process=num_process)
with open(retrosys_smiles_calculate_similarity_filter_file_path, 'r') as json_file:
    retrorules = json.load(json_file) 
retrorules = pd.DataFrame(retrorules)

100%|██████████| 60/60 [00:06<00:00,  9.24it/s]


In [9]:
process_retrorules_and_save(rxndb_path,failedrxn_path,retrorules,heterologous_met_smiles=None,num_process=num_process)

100%|██████████| 20210/20210 [07:09<00:00, 47.07it/s] 


### retrosynthesis result screen

In [10]:
#drop metabolites that are not in the ymdb or yeast8
drop_rxndb(rxndb_path, rxndb_drop_path, total_met_inchikey0_file, num_processes=num_process)
#statistic the number of reactions 
statistic_reaction_num(rxndb_path,rxndb_drop_path)

100%|██████████| 12652/12652 [00:02<00:00, 5558.90it/s]


rxndb: 6741177
rxndb_drop: 120852


In [11]:
#merge the rxndb
rxndb = merge_rxndb(rxndb_drop_path,rxndb_all_path)
rxndb_df = pd.DataFrame(rxndb).transpose()
print(rxndb_df.shape)
rxndb_df = rxndb_df.reset_index().rename(columns={'index':'NO'})


(120852, 9)


In [12]:
# #merge the rxndb
# # rxndb = merge_rxndb(rxndb_path,rxndb_all_path)
# with open(rxndb_all_path,'r') as f:
#     rxndb = json.load(f)
# rxndb_df = pd.DataFrame(rxndb).transpose()
# print(rxndb_df.shape)
# rxndb_df = rxndb_df.reset_index().rename(columns={'index':'NO'})


In [13]:
#get all the metabolites in the rxndb
rxndb_all_smiles = parallel_process_rxn_smiles(rxndb_df, n_splits=100,num_process=num_process)

100%|██████████| 100/100 [00:00<00:00, 160.40it/s]


number of metabolites in RXNDB: 5286


In [14]:
#get all the smiles in the yeast8
yeast8_total_smiles = get_all_smiles_in_model(yeast_model_path,ymdb_output_final)

In [15]:
#calculate the similarity of the metabolites

smile_max_score_pd = calculate_save_smiles_max_score(rxndb_all_smiles, yeast8_total_smiles,rxndb_met_max_score_file,num_processes=num_process)

100%|██████████| 5286/5286 [05:17<00:00, 16.67it/s]


In [16]:
# smile_max_score_pd = pd.read_csv(rxndb_met_max_score_file)

In [17]:

smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smile_max_score_pd, num_processes=num_process, num_iterations=7)
smiles_max_score_tmp_pd.to_csv(rxndb_met_max_score_file,index=None)





100%|██████████| 100/100 [00:03<00:00, 31.91it/s]


Iteration 1 - Current success count: 4608
Iteration 1 - Current success_rxndbid count: 85570


100%|██████████| 100/100 [00:03<00:00, 28.66it/s]


Iteration 2 - Current success count: 4825
Iteration 2 - Current success_rxndbid count: 91047


100%|██████████| 100/100 [00:03<00:00, 29.45it/s]


...

Iteration 5 - Current success count: 4873
Iteration 5 - Current success_rxndbid count: 93289


100%|██████████| 100/100 [00:03<00:00, 28.01it/s]


Iteration 6 - Current success count: 4879
Iteration 6 - Current success_rxndbid count: 93887


100%|██████████| 100/100 [00:03<00:00, 28.39it/s]


Iteration 7 - Current success count: 4880
Iteration 7 - Current success_rxndbid count: 93889
final success 4880
final success_rxndbid 93889


In [18]:
# smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smiles_max_score_tmp_pd, num_processes=num_process, num_iterations=2)
# smiles_max_score_tmp_pd.to_csv(rxndb_met_max_score_file,index=None)

In [19]:
# smiles_max_score_tmp_pd = pd.read_csv(rxndb_met_max_score_file)

In [20]:

save_success_fail_target_smiles(target_smiles_path,smiles_success,YMDB_success_met_smile_file,YMDB_fail_met_smile_file)




success number: 215
fail number: 360


In [21]:
#get all the smiles in the yeast8
yeast8_total_smiles = get_all_smiles_in_model(yeast_model_path,ymdb_output_final)

# consider uptake metabolites
uptake = pd.read_csv(uptake_met_path)
uptake_smiles = uptake['SMILES'].values
uptake_smiles = [normalize_smiles(i) for i in uptake_smiles]
yeast8_total_smiles = yeast8_total_smiles + uptake_smiles

In [22]:
smile_max_score_pd = calculate_save_smiles_max_score(rxndb_all_smiles, yeast8_total_smiles,rxndb_met_max_score_file,num_processes=num_process)
# smiles_max_score_tmp_pd = pd.read_csv(rxndb_met_max_score_uptake_file)

smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smile_max_score_pd, num_processes=num_process, num_iterations=7)
smiles_max_score_tmp_pd.to_csv(rxndb_met_max_score_uptake_file,index=None)





100%|██████████| 100/100 [00:03<00:00, 32.42it/s]


Iteration 1 - Current success count: 4854
Iteration 1 - Current success_rxndbid count: 108032


100%|██████████| 100/100 [00:03<00:00, 27.52it/s]


Iteration 2 - Current success count: 5068
Iteration 2 - Current success_rxndbid count: 111826


100%|██████████| 100/100 [00:03<00:00, 27.28it/s]


...

Iteration 5 - Current success count: 5095
Iteration 5 - Current success_rxndbid count: 112318


100%|██████████| 100/100 [00:03<00:00, 28.52it/s]


Iteration 6 - Current success count: 5097
Iteration 6 - Current success_rxndbid count: 112345


100%|██████████| 100/100 [00:03<00:00, 28.22it/s]


Iteration 7 - Current success count: 5097
Iteration 7 - Current success_rxndbid count: 112345
final success 5097
final success_rxndbid 112345


In [23]:
save_success_fail_target_smiles(target_smiles_path,smiles_success,YMDB_success_met_smile_uptake_file,YMDB_fail_met_smile_uptake_file)


success number: 254
fail number: 321


In [24]:
save_rxndb_to_model(rxndb_df,success_rxndbid,rxndb_to_model_path)


(112345, 10)
(112345, 10)
(112345, 5)
(112345, 5)


In [25]:
smiles_success = load_pickle(YMDB_success_met_smile_file)
smiles_success_uptake = load_pickle(YMDB_success_met_smile_uptake_file)
diff = [x for x in smiles_success_uptake if x not in smiles_success]
uptake_synthesis = [x for x in diff if x not in uptake_smiles]
print(len(uptake_synthesis))

4


### RXNDB gene annotation

In [26]:
data = pd.read_csv(rxndb_to_model_path)
print('data:',data.shape)

data: (112345, 5)


### CLEAN

In [27]:
clean_gene2ec_dict = get_gene2ec_dict_clean(sce_gene_clean_ec)

### DeepECv2

In [28]:
DeepEC_gene2ec_dict = get_gene2ec_dict_DeepEC(DeepEC_path)

In [29]:
clean_deepec_gene2ec_dict = {}
for k,v in clean_gene2ec_dict.items():
    if k in DeepEC_gene2ec_dict:
        clean_deepec_gene2ec_dict[k] = v
clean_deepec_ec2gene_dict = get_ec2gene_dict_clean(clean_deepec_gene2ec_dict)
print(len(clean_deepec_ec2gene_dict))

154


In [30]:
rxndb_with_GPR = rxndb_gene_annotation(rxndb_to_model_path,clean_deepec_ec2gene_dict)
rxndb_with_GPR = rxndb_with_GPR[rxndb_with_GPR['GENE'].apply(lambda x: len(x) > 0)]
rxndb_with_GPR.to_csv(rxndb_GPR_to_model_path,index=None)
print('rxndb with GPR:',rxndb_with_GPR.shape)


rxndb all ec num 148
rxndb with GPR: (112004, 6)


In [31]:
rxndb_with_GPR.head()


,NO,EC number,templateID,rxn_smiles_basic,rxn_smiles_final,GENE
0,rxn1,"[1.1.1, 1.3.1]",MNXR100327_reverse,OCC(O)CO>>O=C(CO)CO,OCC(O)CO.NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)...,"[YLR174W, YHR183W, YJR096W, YNL241C, YIL074C, ..."
1,rxn2,"[1.1.1, 1.3.1]",MNXR100327_reverse,OCC(O)C(O)C(O)C(O)CO>>O=C(CO)C(O)C(O)C(O)CO,OCC(O)C(O)C(O)C(O)CO.NC(=O)c1ccc[n+]([C@@H]2O[...,"[YLR174W, YHR183W, YJR096W, YNL241C, YIL074C, ..."
2,rxn3,"[1.1.1, 1.3.1]",MNXR100327_reverse,OCC(O)C(O)C(O)CO>>O=C(CO)C(O)C(O)CO,OCC(O)C(O)C(O)CO.NC(=O)c1ccc[n+]([C@@H]2O[C@H]...,"[YLR174W, YHR183W, YJR096W, YNL241C, YIL074C, ..."
3,rxn4,"[1.1.1, 1.3.1]",MNXR100327_reverse,O=C([O-])C(O)C(O)C(O)C(O)CO>>O=C([O-])C(O)C(O)...,O=C([O-])C(O)C(O)C(O)C(O)CO.NC(=O)c1ccc[n+]([C...,"[YLR174W, YHR183W, YJR096W, YNL241C, YIL074C, ..."
4,rxn5,"[1.1.1, 1.3.1]",MNXR100327_reverse,O=C(O)C(O)C(O)C(O)C(O)CO>>O=C(O)C(=O)C(O)C(O)C...,O=C(O)C(O)C(O)C(O)C(O)CO.NC(=O)c1ccc[n+]([C@@H...,"[YLR174W, YHR183W, YJR096W, YNL241C, YIL074C, ..."


### rxndb metabolite annotation

In [32]:
#rxndb all metabolites
rxndb_all_smile = get_all_met_smile(rxndb_GPR_to_model_path)


100%|██████████| 112004/112004 [00:02<00:00, 37768.28it/s]

5096


In [33]:
#metabolite comparison
yeast8_total_smiles = get_GEM_all_smiles(yeast_model_path)
smile_max_score = calculate_smile_max_score_annotation(rxndb_all_smile,yeast8_total_smiles,num_process=num_process)
smile_max_score_pd = pd.DataFrame(smile_max_score)
smile_max_score_pd.to_csv(rxndb_met_max_score_met_annotation_file,index=None)


100%|██████████| 5096/5096 [02:09<00:00, 39.26it/s]


In [34]:
#assign ID
yeast8_id_smiles_mapping = get_yeast8_id_smiles_mapping(yeast_model_path)
smile_max_score_pd_with_ID = assign_ID_to_RXNDB_met(smile_max_score_pd,yeast8_id_smiles_mapping,s = 'not_lipid')
metabolites_info_to_GEM = get_new_met_smile_list(smile_max_score_pd_with_ID)
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)



/home/haohao/underground/Underground_workflow/Code/retrosynthesis/not_lipid_retrosynthesis/../common.py:1353: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yeast_id_mapping['smiles'] = yeast_id_mapping['smiles'].apply(normalize_smiles)
/home/haohao/underground/Underground_workflow/Code/retrosynthesis/not_lipid_retrosynthesis/../common.py:1354: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yeast_id_mapping['inchikey0'] = yeast_id_mapping['smiles'].apply(smiles2inchikey0)
100%|██████████| 5096/5096 [00:03<00:

### Generate equation according to metabolite ID

In [35]:
smiles_to_id_mapping = get_smiles_to_id_mapping(metabolites_info_to_GEM_path)
print(len(smiles_to_id_mapping))

5096


In [36]:
rxndb_total_info_to_model = get_rxndb_to_model_total_info(rxndb_GPR_to_model_path,smiles_to_id_mapping)
rxndb_total_info_to_model = rxndb_total_info_to_model[['NO','EC_number','templateID','rxn_smiles_basic','rxn_smiles_final','GPR','equation_dict','equation']]
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)
print(rxndb_total_info_to_model.shape)

(103543, 8)


### check the formula of the model metabolites in the predicted reaction

In [37]:
metabolites_info_to_GEM,rxndb_total_info_to_model = check_met_and_reaction_formula(yeast_model_path,metabolites_info_to_GEM_path,rxndb_total_info_to_model_path)
metabolites_info_to_GEM.to_csv(metabolites_info_to_GEM_path,index=None)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)
print(rxndb_total_info_to_model.shape)

formula error_met set()


0it [00:00, ?it/s]


(103543, 8)


### remove duplicate reactions and existing reactions

In [38]:
same_rxndb_reaction = get_same_rxndb_reaction_mapping(rxndb_total_info_to_model_path)
rxndb_total_info_to_model = get_no_same_rxndb_reaction_parallelize(rxndb_total_info_to_model_path,get_no_same_rxndb_reaction,same_rxndb_reaction,n_cores=num_process)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_path,index=None)
print(rxndb_total_info_to_model.shape)

(103543, 8)


100%|██████████| 103543/103543 [00:03<00:00, 26086.90it/s]


(9377, 8)


In [39]:
met_info = pd.read_csv(metabolites_info_to_GEM_path)
rxndb_to_model_total_info = pd.read_csv(rxndb_total_info_to_model_path)

### Find out the reactions that already exist in the model

In [40]:
with open(miss_met_id_file) as f:
    miss_met_id = json.load(f) 
model = cobra.io.load_yaml_model(yeast870_path)


### yeast8_smiles_list, ID with smiles

In [41]:
model_excel, metID_with_smiles, yeast8_smiles_lst = process_model(yeast_model_path, miss_met_id)


2124
['s_0006', 's_0007', 's_0008', 's_0009', 's_0010', 's_0011', 's_0012', 's_0013', 's_0014', 's_0015', 's_0016', 's_0017', 's_0018', 's_0019', 's_0020', 's_0021', 's_0022', 's_0023', 's_0024', 's_0025', 's_0026', 's_0027', 's_0028', 's_0029', 's_0030', 's_0031', 's_0032', 's_0033', 's_0034', 's_0035', 's_0036', 's_0037', 's_0038', 's_0039', 's_0042', 's_0045', 's_0048', 's_0051', 's_0054', 's_0056', 's_0058', 's_0060', 's_0061', 's_0062', 's_0063', 's_0064', 's_0065', 's_0066', 's_0067', 's_0068', 's_0069', 's_0075', 's_0076', 's_0077', 's_0078', 's_0079', 's_0080', 's_0086', 's_0087', 's_0116', 's_0117', 's_0118', 's_0119', 's_0120', 's_0121', 's_0122', 's_0123', 's_0124', 's_0125', 's_0126', 's_0127', 's_0129', 's_0131', 's_0132', 's_0133', 's_0134', 's_0135', 's_0136', 's_0137', 's_0138', 's_0139', 's_0140', 's_0141', 's_0142', 's_0143', 's_0144', 's_0145', 's_0146', 's_0147', 's_0148', 's_0149', 's_0150', 's_0152', 's_0153', 's_0154', 's_0155', 's_0156', 's_0157', 's_0158', 's_0

In [42]:
rxndb_met_with_s_smiles_lst = get_rxndb_met_with_s_smiles_lst(rxndb_total_info_to_model_path,model_excel,miss_met_id)
reaID_with_smiles_dict = get_reaID_with_smiles_dict(model, model_excel, miss_met_id,rxndb_met_with_s_smiles_lst,metID_with_smiles)


(9377, 8)


100%|██████████| 9377/9377 [00:08<00:00, 1042.43it/s]


821
['NC(CS)C(=O)O', 'NC(CO)C(=O)O', 'CC(=O)NC(CS)C(=O)O', 'CCCCCC(=O)[O-]', 'CCCCCCCCCCCCCCCC(=O)O', 'CCCCCCCCCCCCCC(=O)O', 'CCCCCCCCCCCC(=O)[O-]', 'CCCCCCCCCC(=O)[O-]', 'CCCCCCCC(=O)[O-]', 'CCCCCCCC(=O)OCC']
2702
1184
{'r_0003': ['CC(O)C(C)O', 'CC(=O)C(C)O'], 'r_0007': ['NC(=O)c1ncn(C2OC(COP(=O)([O-])[O-])C(O)C2O)c1N=CNC1OC(COP(=O)([O-])[O-])C(O)C1O', 'NC(=O)c1ncn(C2OC(COP(=O)([O-])[O-])C(O)C2O)c1NC=NCC(=O)C(O)C(O)COP(=O)([O-])[O-]'], 'r_0012': ['NC(CCC(=O)O)C(=O)O', 'O=C(O)C1CCC=N1'], 'r_0013': ['CSCCC(=O)C(=O)O', 'O=C[O-]', 'CSCCC(=O)C(=O)COP(=O)(O)O'], 'r_0014': ['Nc1nc(NCC(O)C(O)C(O)COP(=O)(O)O)c(N)c(=O)[nH]1', 'Nc1c(NCC(O)C(O)C(O)COP(=O)(O)O)[nH]c(=O)[nH]c1=O'], 'r_0015': ['Nc1nc(NCC(O)C(O)C(O)COP(=O)(O)O)c(N)c(=O)[nH]1', 'Nc1nc(NC2OC(COP(=O)(O)O)C(O)C2O)c(N)c(=O)[nH]1'], 'r_0016': ['CCC(=O)C(=O)[O-]', 'CCC(O)(C(C)=O)C(=O)[O-]', 'O=C=O', 'CC(=O)C(=O)[O-]'], 'r_0018': ['NC(CCCC(=O)O)C(=O)O', 'NC(CCC(=O)O)C(=O)O', 'O=C(O)CCC(=O)C(=O)O', 'O=C(O)CCCC(=O)C(=O)O'], 'r_0019': ['CC(C)(C

In [43]:
yeast8_reaction_in_rxndb = get_yeast8_reaction_in_rxndb_parallel(rxndb_total_info_to_model_path,reaID_with_smiles_dict,model_excel,miss_met_id,num_process=num_process)


(9377, 8)
(5908, 8)


In [44]:
with open(yeast8_reaction_in_rxndb_json, 'w') as json_file:
    json.dump(yeast8_reaction_in_rxndb, json_file,indent=4)

In [45]:
get_reaction_recovery_ratio(yeast8_reaction_in_rxndb)
#cut off 0.3
#top20,0.791
#top30,0.795
#top40,0.8
#top50,0.8
#top60,0.806
#top70,0.826
#top80,0.827

0.785472972972973


In [46]:
# yeast8_reaction_in_rxndb = {k:v for k,v in yeast8_reaction_in_rxndb.items() if len(v)>0}



### Check the accuracy of GPR

In [47]:
#Gene2ec was obtained according to uniprot
# sce_gene2ec_dict = get_sce_gene2ec_dict(sce_uniprot_path)


In [48]:
# check_gpr_accuracy(yeast8_reaction_in_rxndb,model,sce_gene2ec_dict,rxndb_to_model_total_info)
#top20,0.849
#top30,0.852
#top40,0.851
#top50,0.854
#top60,0.861
#top70,0.864
#top80,0.871